In [12]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim import corpora
from gensim import models

/Users/mmiyazaki/dataiku/Design/DATA_DIR/code-envs/python/japan-nlp/lib/python3.6/site-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [2]:
# Read recipe inputs
ramen_clusters_named = dataiku.Dataset("ramen_clusters_named")
df = ramen_clusters_named.get_dataframe()

In [3]:
list_vocabs = df[df['cluster_labels'].isin(['接客などのサービス', '具材・素材・味'])]['words_concat'].values

In [4]:
ramen_word = list_vocabs[0].split(",") + list_vocabs[1].split(",")

In [7]:
folder_path = dataiku.Folder("aLTWBozg").get_path()
text_path = folder_path + "/ramen_corpus.txt"

In [9]:
f = open(text_path,'r',encoding="utf-8")
trainings = []

In [10]:
for i,data in enumerate(f):
    word = data.replace("'",'').replace('[','').replace(']','').replace(' ','').replace('\n','').split(",")
    trainings.append([i for i in word if i in ramen_word])

In [31]:
len(trainings)

211

In [13]:
dictionary = corpora.Dictionary(trainings)

In [15]:
corpus = list(map(dictionary.doc2bow,trainings))

In [16]:
test_model = models.TfidfModel(corpus)

In [17]:
corpus_tfidf = test_model[corpus]

In [19]:
# id->単語へ変換
texts_tfidf = [] # id -> 単語表示に変えた文書ごとのTF-IDF
for doc in corpus_tfidf:
    text_tfidf = []
    for word in doc:
        text_tfidf.append([dictionary[word[0]],word[1]])
    texts_tfidf.append(text_tfidf)

In [28]:
from operator import itemgetter

texts_tfidf_sorted_top20 = [] 

#TF-IDF値を高い順に並び替え上位単語20個に絞る。
for i in range(len(texts_tfidf)):
    soted = sorted(texts_tfidf[i], key=itemgetter(1),reverse=True)
    soted_top20 = soted[:20]
    word_list = []
    for k in range(len(soted_top20)):
        word = soted_top20[k][0]
        word_list.append(word)
    texts_tfidf_sorted_top20.append(word_list)

In [0]:
# Compute recipe outputs from inputs
# TODO: Replace this part by your actual code that computes the output, as a Pandas dataframe
# NB: DSS also supports other kinds of APIs for reading and writing data. Please see doc.

tf_IDF_scores_df = ramen_clusters_named_df # For this sample code, simply copy input to output


# Write recipe outputs
tf_IDF_scores = dataiku.Dataset("ramen_by_store_name").get_dataframe()